# Hugging Face Transformers ile Soyutlayıcı Özetleme


## Giriş

Otomatik özetleme, Doğal Dil İşleme (NLP) de ki temel sorunlardan biridir. İnsan dilini bilgisayarlara öğretebilmek için çeşitli zorluk ortaya çıkar.
Anlama (örneğin, önemli içeriği belirleme)
ve oluşturma (örneğin, tanımlanan içeriğin bir özet halinde toplanması ve yeniden ifade edilmesi).

Bu öğreticide, tek belgeli özetleme görevini soyutlayıcı bir modelleme yaklaşımı ile ele alıyoruz

Bu özetleme yaklaşımı *Soyutlayıcı Özetleme* olarak bilinir ve
çeşitli disiplinlerdeki araştırmacılar arasında artan ilgiye sahiptir.

Sequence-to-sequence model kullanarak soyutlayıcı özetleme görevini çözmek istiyoruz. [Text-to-Text Transfer Transformer (T5)](https://arxiv.org/abs/1910.10683), [Transformer-based](https://arxiv.org/abs/1706.03762)  

Bu not defterinde, önceden eğitilmiş T5'in Soyutlayıcı Özetleme üzerinde ince ayarını yapacağız.
Hugging Face Veri Kümelerinden yüklenen `XSum` veri kümesinde Hugging Face Transformers kullanacağız.

## Kurulum

### Gereksinimlerin yüklenmesi

In [ ]:
%%shell
pip install transformers==4.20.0
pip install keras_nlp==0.3.0
pip install datasets
pip install huggingface-hub
pip install nltk

### Gerekli kitaplıklıkarı içeri aktarma.

In [2]:
import os
import logging

import nltk
import numpy as np
import tensorflow as tf
from tensorflow import keras

# Yalnızca hata mesajlarını günlüğe kaydet
tf.get_logger().setLevel(logging.ERROR)

os.environ["TOKENIZERS_PARALLELISM"] = "false"

### Belirli değişkenleri tanımlayın

In [3]:
# Eğitim ve test olarak bölmek istediğiniz veri kümesinin yüzdesi
TRAIN_TEST_SPLIT = 0.1

MAX_INPUT_LENGTH = 1024  # Modele girişin maksimum uzunluğu
MIN_TARGET_LENGTH = 5  # Modele göre çıktının minimum uzunluğu
MAX_TARGET_LENGTH = 128  # Modele göre çıktının maksimum uzunluğu
BATCH_SIZE = 8  # Modelimizi eğitmek için parti boyutu
LEARNING_RATE = 2e-5  # Modelimizi eğitmek için öğrenme oranı
MAX_EPOCHS = 1  # Modeli eğiteceğimiz maksimum dönem sayısı

# Bu notebook, Hugging Face Model Hub'daki t5-small kontrol noktası üzerine inşa edilmiştir.
MODEL_CHECKPOINT = "t5-small"

## Veri setini yükleyelim.

Şimdi [Extreme Summarization (XSum)](https://arxiv.org/abs/1808.08745) dosyasını indireceğiz.
Veri seti, BBC haberlerinden ve beraberindeki tek cümlelik özetlerden oluşur.
Spesifik olarak, her haberin başında bir giriş cümlesi (özet olarak da bilinir) bulunur.
Bu veri kümesinde 226.711 makale var
eğitim (%90, 204.045), doğrulama (%5, 11.332) ve test (%5, 11.334) setlerine bölünmüştür.

Literatürün çoğunu takip ederek, özet değerlendirmesi için Recall-Oriented Understudy kullanıyoruz.
Bu diziden diziye özetleyici özetleme yaklaşımımızı değerlendirmek için (ROUGE) metriği.

Hugging Face Datasets'lerini `load_dataset` ile kullanabilmek kolaydır.

In [4]:
from datasets import load_dataset

raw_datasets = load_dataset("xsum", split="train")

Generating train split:   0%|          | 0/204045 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/11332 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/11334 [00:00<?, ? examples/s]

Dataset xsum downloaded and prepared to /root/.cache/huggingface/datasets/xsum/default/1.2.0/082863bf4754ee058a5b6f6525d0cb2b18eadb62c7b370b095d1364050a52b71. Subsequent calls will reuse this data.


Veri kümesi aşağıdaki alanlara sahiptir:

- **document**: Orijinal BBC haberleri
- **summary**: BBC haberlerinin tek cümlelik özeti
- **id**: Belge özeti çiftinin kimliği

In [5]:
print(raw_datasets)

Dataset({
    features: ['document', 'summary', 'id'],
    num_rows: 204045
})


We will now see how the data looks like:

Datasetimizden bir tane örnek görelim.

In [6]:
print(raw_datasets[0])

{'document': 'The full cost of damage in Newton Stewart, one of the areas worst affected, is still being assessed.\nRepair work is ongoing in Hawick and many roads in Peeblesshire remain badly affected by standing water.\nTrains on the west coast mainline face disruption due to damage at the Lamington Viaduct.\nMany businesses and householders were affected by flooding in Newton Stewart after the River Cree overflowed into the town.\nFirst Minister Nicola Sturgeon visited the area to inspect the damage.\nThe waters breached a retaining wall, flooding many commercial properties on Victoria Street - the main shopping thoroughfare.\nJeanette Tate, who owns the Cinnamon Cafe which was badly affected, said she could not fault the multi-agency response once the flood hit.\nHowever, she said more preventative work could have been carried out to ensure the retaining wall did not fail.\n"It is difficult but I do think there is so much publicity for Dumfries and the Nith - and I totally apprecia

In [7]:
raw_datasets = raw_datasets.train_test_split(
    train_size=TRAIN_TEST_SPLIT, test_size=TRAIN_TEST_SPLIT
)

## Veri Ön İşleme

Bu kısmında standart olarak veri ön işleme yapacağız. Ayrıca modellerimiz sayısal veriler ile çalıştığından dolayı onları tokenleştireceğiz. Bunu `tokenizer` ile rahatça yapabiliriz.

`from_pretrained()` yöntemi, Hugging Face Model Hub'dan bir modelin adını bekler. Bu
daha önce açıklanan MODEL_CHECKPOINT ile tamamen benzer işleve sahiptir.

In [12]:
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained(MODEL_CHECKPOINT)

Beş T5 kontrol noktasından birini kullanıyorsanız girişlerin önüne şunu eklemeliyiz:
"summarize:" 
(model ayrıca çevirebilir ve hangi görevi yaptığını bilmek için öneke ihtiyaç duyar.
gerçekleştirmek zorundadır).

In [13]:
if MODEL_CHECKPOINT in ["t5-small", "t5-base", "t5-large", "t5-3b", "t5-11b"]:
    prefix = "summarize: "
else:
    prefix = ""

Hugging Face Datasets ile uyumlu olan ve ön işleme de bize yardımcı olacak basit bir preprocessing fonksiyonu yazacağız.

- Metin veri kümesini, giriş ve hedefler'e karşılık gelen token kimliklerine göre belirtin.
- Ön eki tokenlere ekleyin
- `token_type_ids`, `attention_mask`, gibi girdiler oluşturulaım.

In [14]:

def preprocess_function(examples):
    inputs = [prefix + doc for doc in examples["document"]]
    model_inputs = tokenizer(inputs, max_length=MAX_INPUT_LENGTH, truncation=True)

    # Hedefler için token oluşturucuyu ayarlayın
    with tokenizer.as_target_tokenizer():
        labels = tokenizer(
            examples["summary"], max_length=MAX_TARGET_LENGTH, truncation=True
        )

    model_inputs["labels"] = labels["input_ids"]

    return model_inputs


Bu işlevi veri kümemizdeki tüm cümle çiftlerine uygulamak için, daha önce oluşturduğumuz veri kümesi nesnemizin map yöntemini kullanırız. Bu, işlevi veri kümesindeki tüm bölmelerin tüm öğelerine uygulayacaktır, böylece eğitim, doğrulama ve test verilerimiz tek bir komutta önceden işlenecektir.

In [15]:
tokenized_datasets = raw_datasets.map(preprocess_function, batched=True)

Map:   0%|          | 0/20404 [00:00<?, ? examples/s]

Map:   0%|          | 0/20405 [00:00<?, ? examples/s]

## Modeli tanımlama

Artık önceden eğitilmiş modeli indirebilir ve üzerinde ince ayar yapabiliriz. Madem görevimiz
diziden diziye (hem giriş hem de çıkış metin dizileridir),
Hugging Face Transformers kitaplığından `TFAutoModelForSeq2SeqLM` sınıfı kullanabiliriz.
`from_pretrained` yöntemi modeli bizim için indirecek ve önbelleğe alacaktır.

`from_pretrained()` yöntemi, Hugging Face Model Hub'dan bir modelin adını bekler.Daha önce bahsedilen, t5-base model kontrol noktasını kullanacağız.

In [16]:
from transformers import TFAutoModelForSeq2SeqLM, DataCollatorForSeq2Seq

model = TFAutoModelForSeq2SeqLM.from_pretrained(MODEL_CHECKPOINT)

Downloading:   0%|          | 0.00/231M [00:00<?, ?B/s]

All model checkpoint layers were used when initializing TFT5ForConditionalGeneration.

All the layers of TFT5ForConditionalGeneration were initialized from the model checkpoint at t5-small.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFT5ForConditionalGeneration for predictions without further training.


Sequence to Sequence modellerini eğitmek için özel bir tür veri toplayıcıya ihtiyacımız var,
Bu, girdileri toplu işteki maksimum uzunluğa getirmekle kalmaz, aynı zamanda
etiketler.
Bu nedenle, Hugging Face Transformers tarafından sağlanan `DataCollatorForSeq2Seq` kullanıyoruz.
`return_tensors='tf'`, `tf.Tensor` ile data almamızı sağlar

In [17]:
from transformers import DataCollatorForSeq2Seq

data_collator = DataCollatorForSeq2Seq(tokenizer, model=model, return_tensors="tf")

Ardından, modelimizi eğiteceğimiz eğitim ve test setlerimizi tanımlıyoruz. Yine Hugging Face veri kümeleri bize ayarlamak adına, veri setimizi entegre etmemize yardımcı olacak `to_tf_dataset` yöntemini sağlar.
Yukarıda tanımlanan "collator" ile veri kümesini istediğimiz kıvama getirebiliriz. Yöntem belirli parametreler bekler:

- **columns**: bağımsız değişkenlerimiz olarak görev yapacak sütunlar
- **batch_size**: eğitim için parti büyüklüğümüz
- **shuffle**: veri setimizi karıştırmak isteyip istemediğimizi
- **collate_fn**: collator function

Ek olarak, ROUGE hesaplamak için nispeten daha küçük bir "jgeneration_dataset" de tanımlarız.

In [18]:
train_dataset = tokenized_datasets["train"].to_tf_dataset(
    batch_size=BATCH_SIZE,
    columns=["input_ids", "attention_mask", "labels"],
    shuffle=True,
    collate_fn=data_collator,
)
test_dataset = tokenized_datasets["test"].to_tf_dataset(
    batch_size=BATCH_SIZE,
    columns=["input_ids", "attention_mask", "labels"],
    shuffle=False,
    collate_fn=data_collator,
)
generation_dataset = (
    tokenized_datasets["test"]
    .shuffle()
    .select(list(range(200)))
    .to_tf_dataset(
        batch_size=BATCH_SIZE,
        columns=["input_ids", "attention_mask", "labels"],
        shuffle=False,
        collate_fn=data_collator,
    )
)

## Modeli derlemeyelim.

Şimdi optimize edicimizi tanımlayıp modeli derleyeceğiz!

In [19]:
optimizer = keras.optimizers.Adam(learning_rate=LEARNING_RATE)
model.compile(optimizer=optimizer)

No loss specified in compile() - the model's internal loss computation will be used as the loss. Don't panic - this is a common way to train TensorFlow models in Transformers! To disable this behaviour please pass a loss argument, or explicitly pass `loss=None` if you do not want your model to compute a loss.


## Modelin Eğitimi ve Değerlendirilmesi

Eğitim sırasında modelimizi anında değerlendirmek için `metric_fn` tanımlayacağız.
Gerçek ve tahminler arasındaki `ROUGE` puanını hesaplayın.

In [27]:
!pip install rouge-score

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [29]:
import keras_nlp

rouge_l = keras_nlp.metrics.RougeL


def metric_fn(eval_predictions):
    predictions, labels = eval_predictions
    decoded_predictions = tokenizer.batch_decode(predictions, skip_special_tokens=True)
    for label in labels:
        label[label < 0] = tokenizer.pad_token_id  # Replace masked label tokens
    decoded_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)
    result = rouge_l(decoded_labels, decoded_predictions)
    # Yalnızca F1 puanını yazdıracağız, diğer toplama metriklerini de kullanabilirsiniz
    result = {"RougeL": result["f1_score"]}

    return result


Artık nihayet modelimizi eğitmeye başlayabiliriz!

In [ ]:
from transformers.keras_callbacks import KerasMetricCallback

metric_callback = KerasMetricCallback(
    metric_fn, eval_dataset=generation_dataset, predict_with_generate=True
)

callbacks = [metric_callback]

# Şimdilik test setimizi doğrulama_verilerimiz olarak kullanacağız
model.fit(
    train_dataset, validation_data=test_dataset, epochs=MAX_EPOCHS, callbacks=callbacks
)

En iyi sonuçları elde etmek için, modelin tamamında en az 5 dönem boyunca eğitilmesini öneririz.

## Çıkarım

Şimdi keyfi bir makale üzerinde eğittiğimiz modeli çıkarmaya çalışacağız. Böyle yaparak,
Hugging Face Transformers'daki `pipeline` yöntemini kullanacağız.

`pipeline` yöntemi, eğitilen modeli ve tokenizer'i bağımsız değişken olarak alır.
`framework="tf"` argümanı, TF ile eğitilmiş bir modelden geçmenizi sağlar.

In [ ]:
from transformers import pipeline

summarizer = pipeline("summarization", model=model, tokenizer=tokenizer, framework="tf")

summarizer(
    raw_datasets["test"][0]["document"],
    min_length=MIN_TARGET_LENGTH,
    max_length=MAX_TARGET_LENGTH,
)

Artık bu modeli Hugging Face Model Hub'a aktarabilir ve ayrıca tüm arkadaşlarınız, aileniz ve en sevdiğiniz kişiler ile  paylaşabilirsiniz:
`"your-username/the-name-you-picked"` ile yükleyebilirsiniz. örneğin;

```python
model.push_to_hub("transformers-qa", organization="keras-io")
tokenizer.push_to_hub("transformers-qa", organization="keras-io")
```
Ve modelinizi yeterince test edildikten sonra gelecekte onu bu şekilde yükleyebilirsiniz!

```python
from transformers import TFAutoModelForSeq2SeqLM

model = TFAutoModelForSeq2SeqLM.from_pretrained("your-username/my-awesome-model")
```